# (Generalized) Pattern Lattice Builder (PLB) runner on bare words

developed by Kow Kuroda (kow.kuroda@gmail.com)

Creation
- 2024/11/01

Modifications
- 2024/11/02, 11
- 2025/09/17 renamed to gPLB-runner-on-bare-items.ipynb;
- 2025/10/17 updated for revised version of gPLB with p_metric;

In [ ]:
## imports
import gPLB as plb

## The following needs to be True only for debugging/development.
re_import_module = True
if re_import_module:
    import importlib
    importlib.reload (plb)

In [ ]:
## inspect plb.py
import pprint as pp
content_check = False
if content_check:
    pp.pprint(dir(plb))

In [ ]:
## Parameters

## internal variables
comment_escapes  = ['#', '%']

## parameters for PLB
gap_marks       = [ "_", "…" ]
gap_mark        = gap_marks[0]
tracer          = "~"

## managemental
verbose = True

## if use multiprocess module
use_mp = True

## if Generalized Pattern Lattice is used nor not
generalities = [0,1,2,3]
generality   = generalities[2]

## if instantiation check is reflexive: effective under generalized
reflexive       = True

## if node pooling is reductive
reductive       = True

## key
p_metrics = ['rank', 'gap_size', 'size']
p_metric  = p_metrics[0]

## if robust Z-score is used instead of normal Z-score
use_robust_zscore   = True

## select the bases of z-scores: link sources or link targets
zscores_from_targets = False

### settings for drawing
## draw individual lattices
draw_individual_lattices   = False
draw_sample_lattices_only  = True
sample_lattice_n           = 3

## full display for merged pattern lattice
draw_full_version_for_merged = False

## Graph drawing parameters
layout = "Multi_partite"

## key of Multi-partite Graph (MPG) layout
MPG_keys = ['gap_size', 'rank', 'size']
MPG_key  = MPG_keys[0]

## settings for CJK font display
## N.B. phonetic symbols are likely to be incompatible with CJK-ready font
use_multibyte_chars = False

In [ ]:
## parameters for data sampling
min_len   = 3
max_len   = 5
sample_n  = 9

In [ ]:
## set up font
import matplotlib
from matplotlib import font_manager as Font_manager

if use_multibyte_chars:
    ## select font
    multibyte_font_names = [    "IPAexGothic",  # 0 Multi-platform font
                                "Hiragino sans" # 1 Mac only
                            ]
    multibyte_font_name  = multibyte_font_names[0]
    ## tell where target fonts are
    system_font_dir = "/System/Library/Fonts/"
    user_font_dir = "/Library/Fonts/"
    # use the version installed via TeXLive
    user_font_dir2 = "/usr/local/texlive/2013/texmf-dist/fonts/truetype/public/ipaex/"
    if multibyte_font_name == "IPAexGothic":
        try:
            Font_manager.fontManager.addfont(f"{user_font_dir}ipaexg.ttf")
        except FileNotFoundError:
            Font_manager.fontManager.addfont(f"{user_font_dir2}ipaexg.ttf")
    elif multibyte_font_name == "Hiragino sans":
        Font_manager.fontManager.addfont(f"{system_font_dir}ヒラギノ角ゴシック W0.ttc")
    ## check result
    matplotlib.rc('font', family = multibyte_font_name)
else:
    multibyte_font_name = None
    matplotlib.rcParams['font.family'] = "Sans-serif"
##
print(f"multibyte_font_name: {multibyte_font_name}")
print(f"matplotlib.rcParams['font.family']: {matplotlib.rcParams['font.family']}")    

In [ ]:
## graph layout selection
graph_layouts = [   'Multi_partite', # 0
                    'Graphviz', # 1
                    'ARF', # 2
                    'Fruchterman_Reingold', # 3
                    'Kamada_Kawai', # 4
                    'Spring', # 5
                    'Planar', # 6
                    'Circular', # 7
                    'Shell', # 8
                    'Planar', # 9 [often fails to work]
                    # 'Breadth-First Search', # fails work
                 ]
#
graph_layout = graph_layouts[0]
print(f"graph_layout: {graph_layout}")

# Preparation

In [ ]:
## select target file
from pathlib import Path
wd = Path("./data-raw-words")
data_files = list(wd.glob('*.txt'))
data_file  = data_files[0]
print(f"read data from: {data_file}")

In [ ]:
if str(data_file).endswith(".csv"):
    field_seps          = r",;"
    sep2_is_suppressive = True
else:
    field_seps = ""
    sep2_is_suppressive = False
print(f"field_seps: '{field_seps}'")

In [ ]:
## reading data
with open(data_file, encoding = "utf-8") as file:
    lines = [ line.strip() for line in file.readlines() if len(line) > 0 ]

## discard comment lines that start with #
lines = [ plb.strip_comment(line, comment_escapes) for line in lines ]
print(lines)

In [ ]:
## select targets by pattern matching
import re
selector = r".*f.*a.*n.*"
filtered_lines = [ w for w in lines if re.match(selector, w) ]
filtered_lines

In [ ]:
## filtering data
filtered_lines = [ line for line in filtered_lines if len(line) >= min_len and len(line) <= max_len ]
filtered_lines

In [ ]:
## sampling
import random
sampled_lines = random.sample(filtered_lines, sample_n)
sampled_lines
print("sampled lines")
pp.pprint(sampled_lines)

In [ ]:
## convert words into lists
import re
data = [ [ y for y in re.split(field_seps, x) if len(y) > 0 ] for x in sampled_lines ]
		 
## check result
import pprint as pp
pp.pprint(data)

In [ ]:
# Process Patter Lattice incrementally
lattice_build_is_stepwise = True

In [ ]:
## build pattern lattices from data
gen_links_internally = False # Cruciall for memory conservation

null_pat = plb.Pattern([], gap_mark, tracer)
M = plb.PatternLattice(null_pat, generality = generality, p_metric = p_metric, check = False)
for i, item in enumerate(data):

    print(f"##building PatternLattice {i+1} from {item}")
    pat    = plb.Pattern(item, gap_mark, tracer)
    patlat = plb.PatternLattice(pat, generality = generality, reflexive = reflexive)
    if verbose:
        print(patlat)
    if draw_individual_lattices:
        patlat.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, auto_figsizing = True, generality = generality, zscores_from_targets = zscores_from_targets, font_name = multibyte_font_name)
    
    ## Merging pattern lattices
    print(f"##merging PatternLattices {i} and {i+1}")
    M = M.merge_with (patlat, gen_links_internally = gen_links_internally, generality = generality, reflexive = reflexive, reductive = reductive, use_mp = use_mp, check = False)
    
    ## Delete used patlat
    patlat = None

    print(f"##finished step {i+1}")
    print(f"=================================\n")

In [ ]:
## The following process was isolated for memory conservation
if not gen_links_internally and len(M.links) == 0:
    print(f"##start generating links in delayed manner")
    M.update_links (reflexive = reflexive, check = False)

In [ ]:
print(f"p_metric: {p_metric}")

In [ ]:
## Calculate
plb.gen_zscores_from_targets_by (p_metric, M, gap_mark = gap_mark, tracer = tracer, use_robust_zscore = use_robust_zscore, check = False)
plb.gen_zscores_from_sources_by (p_metric, M, gap_mark = gap_mark, tracer = tracer, use_robust_zscore = use_robust_zscore, check = False)

In [ ]:
## select z-score base
print(f"#zscores_from_targets: {zscores_from_targets}")
if zscores_from_targets:
    if use_robust_zscore:
        Link_bases = M.robust_zscores_from_targets
        print(f"robust z-scores are obtained from link_targets")
    else:
        Link_bases = M.zscores_from_targets
        print(f"normal z-scores are obtained from link_targets")
else:
    if use_robust_zscore:
        Link_bases = M.robust_zscores_from_sources
        print(f"robust z-scores are obtained from link_sources")
    else:
        Link_bases = M.zscores_from_sources
        print(f"normal z-scores are obtained from link_sources")

In [ ]:
## check result
i = 0
for key, value in Link_bases.items():
    i += 1
    print(f"node {i:3d} {key} has z-score: {value: .4f}")

In [ ]:
## draw a pruned diagram from the merged lattice
graph_layouts = [   'Multi_partite', # 0
                    'Graphviz', # 1
                    'ARF', # 2
                    'Fruchterman_Reingold', # 3
                    'Kamada_Kawai', # 4
                    'Spring', # 5
                    'Shell', # 6
                    'Spectral', # 7
                    'Circular', # 8
                    'Planar' # 9 # often fails
                 ]
##
layout             = graph_layouts[0]
scale_factor       = 3
zscore_lowerbound  = 0
zscore_upperbound  = None

In [ ]:
graph_layouts

In [ ]:
## Draw a graph of Pattern Lattice
label_sample_n = 7
for layout in graph_layouts:
   if layout == 'Multi_partite':
      set_figsize_manually = False
      if set_figsize_manually:
         M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, auto_figsizing = False, fig_size = (12, 20), generality = generality, p_metric = p_metric, zscores_from_targets = zscores_from_targets, label_sample_n = label_sample_n, label_size = 12, node_size = 30, font_name = multibyte_font_name, zscore_lb = zscore_lowerbound, zscore_ub = zscore_upperbound, scale_factor = scale_factor, check = False)
      else:
         M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, auto_figsizing = True, generality = generality, p_metric = p_metric, zscores_from_targets = zscores_from_targets, label_sample_n = label_sample_n, zscore_lb = zscore_lowerbound, zscore_ub = zscore_upperbound, scale_factor = scale_factor, font_name = multibyte_font_name, check = False)
   else:
      try:
         M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, auto_figsizing = True, generality = generality, p_metric = p_metric, zscores_from_targets = zscores_from_targets, label_sample_n = label_sample_n, zscore_lb = zscore_lowerbound, zscore_ub = zscore_upperbound, scale_factor = scale_factor, font_name = multibyte_font_name, check = False)
      except NetworkXException as e:
         print(f"layout {layout} failed: {e}")

NameError: name 'graph_layouts' is not defined

In [ ]:
if lattice_build_is_stepwise:
	raise "StopExecution"

# Process individual pattern lattices

In [ ]:
## build pattern lattices from data
Lx = [ ]
for item in data:
    pat    = plb.Pattern(item, gap_mark, tracer)
    patlat = plb.PatternLattice(pat, generality, p_metric, reflexive = reflexive)
    if verbose:
            print(patlat)
    Lx.append(patlat)

In [ ]:
## draw pattern lattices individually
if draw_individual_lattices:
    if draw_sample_lattices_only:
        Ly = random.sample(Lx, sample_lattice_n)
    else:
        Ly = Lx
    for patlat in Ly:
        patlat.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, auto_figsizing = True, generality = generality, font_name = multibyte_font_name)

# Process merged pattern lattice

In [ ]:
## merge pattern lattices
import functools
gen_links_internally = False # Cruciall for memory conservation
M = functools.reduce (lambda La, Lb: La.merge_with (Lb, gen_links_internally = gen_links_internally, generality = generality, reductive = reductive, reflexive = reflexive, use_mp = use_mp, show_steps = True, check = False),
                                                        Lx)

# The following process was isolated for memory conservation
if not gen_links_internally and len(M.links) == 0:
    print(f"##runnning delayed link generation ...")
    M.update_links (reflexive = reflexive, check = False)

In [ ]:
## check robust z-score distribution
import matplotlib.pyplot as plt
plt.hist(M.robust_source_zscores, bins = 20)
plt.title(f"Distribution of robust z-scores for out-going link counts")
plt.show()

In [ ]:
## check robust z-score distribution
import matplotlib.pyplot as plt
plt.hist(M.normal_source_zscores, bins = 20)
plt.title(f"Distribution of (unused) z-scores for out-going link counts")
plt.show()

In [ ]:
## check z-score distribution
import matplotlib.pyplot as plt
plt.hist(M.target_robust_zscores, bins = 20)
plt.title(f"Distribution of robust z-scores for in-coming link counts")
plt.show()

In [ ]:
## check z-score distribution
import matplotlib.pyplot as plt
plt.hist(M.target_zscores, bins = 20)
plt.title(f"Distribution of (unused) z-scores for in-coming link counts")
plt.show()

In [ ]:
## select z-score base
if zscores_from_targets:
    Link_bases = M.source_zscores
else:
    Link_bases = M.target_zscores
## check result
i = 0
for key, value in Link_bases.items():
    i += 1
    print(f"node {i:3d} {key} has z-score: {value: .4f}")

In [ ]:
## draw a pruned diagram from the merged lattice
graph_layouts = [   'Multi_partite', # 0
                    'Graphviz', # 1
                    'ARF', # 2
                    'Fruchterman_Reingold', # 3
                    'Kamada_Kawai', # 4
                    'Spring', # 5
                    'Shell', # 6
                    'Spectral', # 7
                    'Circular', # 8
                    'Planar' # 9 # often fails
                 ]
##
layout             = graph_layouts[0]
scale_factor       = 3
zscore_lowerbound  = -0.5

manually_set = True
if manually_set:
   M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, generality = generality, auto_fig_sizing = False, fig_size = (10, 15), label_sample_n = 4, label_size = 16, node_size = 52, font_name = multibyte_font_name, zscore_lowerbound = zscore_lowerbound, scale_factor = scale_factor, check = False)
else:
   M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, generality = generality, auto_fig_sizing = True, label_sample_n = 4, zscore_lowerbound = zscore_lowerbound, scale_factor = scale_factor, font_name = multibyte_font_name, check = False)

In [ ]:
## draw a full diagram from the merged lattice
if draw_full_version_for_merged:
    layout = graph_layouts[0]
    zscore_lowerbound  = None
    scale_factor       = 3
    ##
    adjusted = False
    if adjusted:
        M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, zscore_lowerbound = zscore_lowerbound,
        scale_factor = scale_factor, fig_size = (24, 60), label_size = 12, node_size = 33, font_name = multibyte_font_name, check = False)
    else:
        M.draw_lattice (layout, MPG_key, save_lattice = False, draw_inline = True, zscore_lowerbound = zscore_lowerbound, scale_factor = scale_factor, font_name = multibyte_font_name, check = False)